In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
#!conda create --name fair_env python=3.7 -y
#!conda activate fair_env

In [3]:
pip install -r requirements.txt

  Using cached cycler-0.10.0-py2.py3-none-any.whl.metadata (722 bytes)
  Using cached decorator-4.0.11-py2.py3-none-any.whl.metadata (2.2 kB)
  Using cached matplotlib-2.0.0.tar.gz (53.2 MB)
  Preparing metadata (setup.py) ... done
  Using cached networkx-1.11-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached numpy-1.14.0.zip (4.9 MB)
  Preparing metadata (setup.py) ... done
  Using cached pandas-0.21.1.tar.gz (11.3 MB)
  Installing build dependencies ... |^C
anceled
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [68]:
!python --version

Python 3.7.16


In [2]:
!pip3 install fairness

In [9]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:
import fairness
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn import datasets as ds

from fairness.algorithms.zafar.ZafarAlgorithm import ZafarAlgorithmBaseline, ZafarAlgorithmAccuracy, ZafarAlgorithmFairness
from fairness.algorithms.kamishima.KamishimaAlgorithm import KamishimaAlgorithm
from fairness.algorithms.kamishima.CaldersAlgorithm import CaldersAlgorithm
from fairness.algorithms.feldman.FeldmanAlgorithm import FeldmanAlgorithm
from fairness.algorithms.baseline.SVM import SVM
from fairness.algorithms.baseline.DecisionTree import DecisionTree
from fairness.algorithms.baseline.GaussianNB import GaussianNB
from fairness.algorithms.baseline.LogisticRegression import LogisticRegression
from fairness.algorithms.ParamGridSearch import ParamGridSearch
from fairness.algorithms.Ben.SDBSVM import SDBSVM


def display_results(y_pred, y, sensitive):
    y_pred2 = (y_pred>0.5).astype(int)
    accuracy = accuracy_score(y, np.squeeze(y_pred2))
    print("Accuracy:", accuracy)
    print("PRULE : ", p_rule(y_pred,sensitive))
    print("DI : ", DI(y_pred, sensitive))
    print("DispFPR : ", DispFPR(y_pred2, y, sensitive))
    print("DispFNR : ", DispFNR(y_pred2, y, sensitive))
    return {'Accuracy': accuracy, 'PRULE': p_rule(y_pred,sensitive), 'DispFPR': DispFPR(y_pred2, y, sensitive)
            ,'DispFNR': DispFNR(y_pred2, y, sensitive)}

def p_rule(y_pred, z_values, threshold=0.5):
    
    y_z_1 = y_pred[z_values == 1] > threshold if threshold else y_pred[z_values == 1]
    y_z_0 = y_pred[z_values == 0] > threshold if threshold else y_pred[z_values == 0]
    odds = y_z_1.mean() / y_z_0.mean()
    return np.min([odds, 1/odds]) * 100

def DI(y_pred, z_values, threshold=0.5):
    y_z_1 = y_pred[z_values == 1] > threshold if threshold else y_pred[z_values == 1]
    y_z_0 = y_pred[z_values == 0] > threshold if threshold else y_pred[z_values == 0]
    odds = abs(y_z_1.mean() - y_z_0.mean())
    return odds
   
def DispFNR(y_pred, y, z_values, threshold=0.5):
    ypred_z_1 = y_pred > threshold if threshold else y_pred[z_values == 1]
    ypred_z_0 = y_pred > threshold if threshold else y_pred[z_values == 0]
    result=abs(ypred_z_1[(y==1) & (z_values==0)].mean()-ypred_z_1[(y==1) & (z_values==1)].mean())
    return result
def DispFPR(y_pred, y, z_values, threshold=0.5):
    ypred_z_1 = y_pred > threshold if threshold else y_pred[z_values == 1]
    ypred_z_0 = y_pred > threshold if threshold else y_pred[z_values == 0]
    result=abs(ypred_z_1[(y==0) & (z_values==0)].mean()-ypred_z_1[(y==0) & (z_values==1)].mean())
    return result


from fairness.data.objects.list import DATASETS, get_dataset_names
from fairness.data.objects.ProcessedData import ProcessedData
from sklearn.preprocessing import StandardScaler, MaxAbsScaler
from fairness.benchmark import run_alg
from sklearn.metrics import accuracy_score
# In[2]:

    
def run_alg(algorithm,X_train,y_train, S_train,X_test,y_test, S_test,params):
    sensitive='S'
    X_train_temp =  pd.DataFrame(X_train)
    X_train_temp['Y'] = y_train.values
    X_train_temp['S']= S_train

    X_test_temp =  pd.DataFrame(X_test)
    X_test_temp['Y'] = y_test.values
    X_test_temp['S']= S_test
    all_sensitive_attributes = ['S']
    ycible = 'Y'
    positive_val=1
    privileged_vals=[1, 1, '1-1']
    class_attr = ycible
    #params = {'lambda': 0.8} 
    #params = algorithm.get_default_params()
    predictions, predictions_list =  \
        algorithm.run(X_train_temp, X_test_temp, class_attr, positive_val, all_sensitive_attributes,
                      sensitive, privileged_vals, params)        
    return predictions, params, predictions_list

#get_ipython().run_line_magic('load_ext', 'autoreload')
#get_ipython().run_line_magic('autoreload', '2')
# In[3]:


import sys
import os
sys.path.insert(1, os.path.join(sys.path[0], '..'))

from models.zhang import ZHANG
from models.adel import ADEL
from models.grari import GRARI

#from models.Broad import BROAD

from data.load_fairness_data import LOAD_DATA_TRAIN_TEST
#

X_train0, X_test0,X_train, X_test, y_train, y_test, S_train, S_test, column_names = LOAD_DATA_TRAIN_TEST('compass')

X_train.shape, X_test.shape


from torch import nn
import torch


class NN_y_adel(nn.Module):
    def __init__(self):
        super(NN_y_adel, self).__init__()
        self.fc1 = nn.Linear(taille_z, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        return x
    
    
class NN_z_adel(nn.Module):
    def __init__(self):
        super(NN_z_adel, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, taille_z)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        return x

    
class NN_s_adel(nn.Module):
    def __init__(self):
        super(NN_s_adel, self).__init__()
        self.fc1_s = nn.Linear(taille_z, 64)
        self.fc2_s = nn.Linear(64, 32)
        self.fc3_s = nn.Linear(32, 16)
        self.fc4_s = nn.Linear(16, 1)

    def forward(self, x):
        x = self.fc1_s(x)
        x = torch.relu(x)
        x = self.fc2_s(x)
        x = torch.relu(x)
        x = self.fc3_s(x)
        x = torch.relu(x)
        x = self.fc4_s(x)
        return x

taille_z = 64
H = 15
H2 = 15

class Net_HGR(nn.Module):
    def __init__(self):
        super(Net_HGR, self).__init__()
        self.fc1 = nn.Linear(1, H)
        self.fc2 = nn.Linear(H, H)
        self.fc3 = nn.Linear(H, H2)
        self.fc4 = nn.Linear(H2, 1)
        self.bn1 = nn.BatchNorm1d(1)

    def forward(self, x):
        h1 = torch.tanh(self.fc1(x))
        h2 = torch.relu(self.fc2(h1))
        h3 = torch.tanh(self.fc3(h2))
        h4 = torch.tanh(self.fc4(h3))
        return h4    
class Net2_HGR(nn.Module):
    def __init__(self):
        super(Net2_HGR, self).__init__()
        self.fc1 = nn.Linear(1, H)
        self.fc2 = nn.Linear(H, H)
        self.fc3 = nn.Linear(H, H2)
        self.fc4 = nn.Linear(H2, 1)
        self.bn1 = nn.BatchNorm1d(1)

    def forward(self, x):
        h1 = torch.tanh(self.fc1(x))
        h2 = torch.relu(self.fc2(h1))
        h3 = torch.tanh(self.fc3(h2))
        h4 = torch.tanh(self.fc4(h3))
        return h4    
class NN_y(nn.Module):
    def __init__(self):
        super(NN_y, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        return x

class NN_s(nn.Module):
    def __init__(self):
        super(NN_s, self).__init__()
        self.fc1_s = nn.Linear(1, 64)
        self.fc2_s = nn.Linear(64, 32)
        self.fc3_s = nn.Linear(32, 16)
        self.fc4_s = nn.Linear(16, 1)

    def forward(self, x):
        x = self.fc1_s(x)
        x = torch.relu(x)
        x = self.fc2_s(x)
        x = torch.relu(x)
        x = self.fc3_s(x)
        x = torch.relu(x)
        x = self.fc4_s(x)
        return x



taille_z = 64



class NN_y_dro(nn.Module):
    def __init__(self):
        super(NN_y_dro, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        return x
            

class NN_s_dro(nn.Module):
    def __init__(self):
        super(NN_s_dro, self).__init__()
        self.fc1_s = nn.Linear(1, 64)
        self.fc2_s = nn.Linear(64, 32)
        self.fc3_s = nn.Linear(32, 16)
        self.fc4_s = nn.Linear(16, 1)

    def forward(self, x):
        x = self.fc1_s(x)
        x = torch.relu(x)
        x = self.fc2_s(x)
        x = torch.relu(x)
        x = self.fc3_s(x)
        x = torch.relu(x)
        x = self.fc4_s(x)
        return x


class NN_a_dro(nn.Module):
    def __init__(self):
        super(NN_a_dro, self).__init__()
        self.fc1 = nn.Linear(X_train.shape[1]+1, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = torch.exp(self.fc3(x))
        return x 

# In[8]:
# # loop results
# In[15]:
# In[ ]:
#LAMBDAS = 
# In[ ]:
#gr[0]['age_q'].unique()
#gr_rdm = get_subgroups_random(X_test0, S_test, n_groups=20)
#dro_scores_rdm = get_scores(gr_rdm[0], X_test0, y_test, S_test, y_pred>0.5, n_min=N_MIN_GROUP,
#          topK_DI=[1, 3], q_DI=10, topK_acc=[1], q_acc=4)

### MULTIPLE RUNS
from utils.fairness_utils import get_subroups_results, get_scores, get_subroups, get_subgroups_random

import warnings
warnings.filterwarnings("ignore")


scores = []
scores_rdm = []

N_RUNS = 2

N_MIN_GROUP = 50
N_EPOCHS = 200
#N_MIN_GROUPS = [10, 20, 30, 50, 80, 100, 200]

import warnings
warnings.filterwarnings("ignore")


Dropping columns: ['two_year_recid', 'sex-race', 'race']


# BIASED MODEL

In [103]:
lambda_=0
dev= "cuda:0"
model = ZHANG(learning_rate=0.0001,batch_size= 2048,
            lamb= lambda_, num_epochs=500,
            NN_y= NN_y, NN_s= NN_s, GPU=dev)
model.fit(X_train, y_train, S_train)
y_pred = model.predict(X_test)
torch.save(model,'./saved_models/NN_biased')

epoch 0 loss 0.58447105 lossS 0.543302 lossY 0.58447105
epoch 5 loss 0.7631674 lossS 0.43635872 lossY 0.7631674
epoch 10 loss 0.5127406 lossS 0.41992107 lossY 0.5127406
epoch 15 loss 0.3726613 lossS 0.4122174 lossY 0.3726613
epoch 20 loss 0.2641065 lossS 0.46515456 lossY 0.2641065
epoch 25 loss 0.36926273 lossS 0.5220292 lossY 0.36926273
epoch 30 loss 0.31710228 lossS 0.28661984 lossY 0.31710228
epoch 35 loss 0.4039368 lossS 0.35104212 lossY 0.4039368
epoch 40 loss 0.3017307 lossS 0.33856896 lossY 0.3017307
epoch 45 loss 0.31994596 lossS 0.26156163 lossY 0.31994596
epoch 50 loss 0.22999987 lossS 0.25316167 lossY 0.22999987
epoch 55 loss 0.2686703 lossS 0.30242166 lossY 0.2686703
epoch 60 loss 0.29717755 lossS 0.23102637 lossY 0.29717755
epoch 65 loss 0.17197673 lossS 0.27342683 lossY 0.17197673
epoch 70 loss 0.21156605 lossS 0.12166057 lossY 0.21156605
epoch 75 loss 0.15525272 lossS 0.30179495 lossY 0.15525272
epoch 80 loss 0.22381237 lossS 0.28821796 lossY 0.22381237
epoch 85 loss 0.2

In [104]:
model=torch.load('./saved_models/NN_biased')
y_pred = model.predict(X_test)
Res = display_results(y_pred, y_test, S_test)
gr = get_subroups(X_test0, S_test, continuous_names=["age"], how_continuous='bins', n_min=N_MIN_GROUP, n_q=10,
             to_keep=["age"]+["sex"])
from utils.fairness_utils import get_subroups_results, get_scores, get_subroups, get_subgroups_random
dro_scores, sr = get_scores(gr[0], X_test0, y_test, S_test, y_pred>0.5, n_min=N_MIN_GROUP,
              topK_DI=[1, 3], q_DI=10, topK_acc=[1], q_acc=4)
print(dro_scores)
sr.drop(["group_label","Acc by S"], axis=1)[['n_obs','sex','age_q','Local DI','Local Acc']]

Accuracy: 0.6473735408560312
PRULE :  67.63537677809592
DI :  0.14754813403610828
DispFPR :  0.0833435394978567
DispFNR :  0.17585147343921287
coucou
{'Global Acc': 0.6473735408560312, 'Global DI': 0.14754813403610828, 'top1_DI': 0.171, 'top3_DI': 0.15066666666666667, 'worst1_acc': 0.593, 'q_DI_0.0': 0.027, 'q_DI_0.1': 0.0282, 'q_DI_0.2': 0.033600000000000005, 'q_DI_0.30000000000000004': 0.047400000000000005, 'q_DI_0.4': 0.07360000000000001, 'q_DI_0.5': 0.106, 'q_DI_0.6000000000000001': 0.1084, 'q_DI_0.7000000000000001': 0.12220000000000002, 'q_DI_0.8': 0.15880000000000005, 'q_DI_0.9': 0.171, 'Local DI': array([0.027, 0.171, 0.029, 0.052, 0.106, 0.11 , 0.171])}


,n_obs,sex,age_q,Local DI,Local Acc
0,181,0,"(20, 30]",0.027,0.646
1,105,0,"(30, 40]",0.171,0.638
2,60,0,"(40, 50]",0.029,0.717
3,798,1,"(20, 30]",0.052,0.629
4,403,1,"(30, 40]",0.106,0.593
5,223,1,"(40, 50]",0.110,0.700
6,148,1,"(50, 60]",0.171,0.757


# ZHANG MODEL

In [105]:
lambda_=25
dev= "cuda:1"
model = ZHANG(learning_rate=0.0001,batch_size= 512,
            lamb= lambda_, num_epochs=200,
            NN_y= NN_y, NN_s= NN_s, GPU=dev)
model.fit(X_train, y_train, S_train)
y_pred = model.predict(X_test)
torch.save(model,'./saved_models/ZHANG_'+ str(lambda_))

epoch 0 loss -7.180868 lossS 0.29795024 lossY 0.26788756
epoch 5 loss -2.9146426 lossS 0.122648284 lossY 0.15156452
epoch 10 loss -5.497427 lossS 0.22781236 lossY 0.19788231
epoch 15 loss -3.3106139 lossS 0.14360671 lossY 0.27955383
epoch 20 loss -5.2962203 lossS 0.2221609 lossY 0.2578027
epoch 25 loss -7.0987964 lossS 0.29082027 lossY 0.1717105
epoch 30 loss -4.8023133 lossS 0.1999895 lossY 0.19742396
epoch 35 loss -4.800734 lossS 0.20051466 lossY 0.21213259
epoch 40 loss -6.070021 lossS 0.24961819 lossY 0.17043363
epoch 45 loss -5.3755975 lossS 0.22136173 lossY 0.158446
epoch 50 loss -7.55431 lossS 0.30988643 lossY 0.19285063
epoch 55 loss -6.8925304 lossS 0.28358075 lossY 0.19698821
epoch 60 loss -6.965284 lossS 0.28615013 lossY 0.1884693
epoch 65 loss -6.9775863 lossS 0.28588325 lossY 0.16949499
epoch 70 loss -5.851677 lossS 0.24322213 lossY 0.2288768
epoch 75 loss -5.4059415 lossS 0.22302023 lossY 0.16956404
epoch 80 loss -4.8130646 lossS 0.20086868 lossY 0.20865238
epoch 85 loss 

In [106]:
model=torch.load('./saved_models/ZHANG_25')
y_pred = model.predict(X_test)
Res = display_results(y_pred, y_test, S_test)
gr = get_subroups(X_test0, S_test, continuous_names=["age"], how_continuous='bins', n_min=N_MIN_GROUP, n_q=10,
             to_keep=["age"]+["sex"])
from utils.fairness_utils import get_subroups_results, get_scores, get_subroups, get_subgroups_random
dro_scores, sr = get_scores(gr[0], X_test0, y_test, S_test, y_pred>0.5, n_min=N_MIN_GROUP,
              topK_DI=[1, 3], q_DI=10, topK_acc=[1], q_acc=4)
print(dro_scores)
sr.drop(["group_label","Acc by S"], axis=1)[['n_obs','sex','age_q','Local DI','Local Acc']]

Accuracy: 0.6352140077821011
PRULE :  82.94305621038295
DI :  0.06486443412952964
DispFPR :  0.0007960808328230373
DispFNR :  0.09851549041737989
coucou
{'Global Acc': 0.6352140077821011, 'Global DI': 0.06486443412952964, 'top1_DI': 0.19, 'top3_DI': 0.13, 'worst1_acc': 0.607, 'q_DI_0.0': 0.007, 'q_DI_0.1': 0.0088, 'q_DI_0.2': 0.016800000000000006, 'q_DI_0.30000000000000004': 0.03720000000000001, 'q_DI_0.4': 0.048400000000000006, 'q_DI_0.5': 0.055, 'q_DI_0.6000000000000001': 0.067, 'q_DI_0.7000000000000001': 0.085, 'q_DI_0.8': 0.11500000000000003, 'q_DI_0.9': 0.15100000000000002, 'Local DI': array([0.075, 0.044, 0.055, 0.01 , 0.007, 0.125, 0.19 ])}


,n_obs,sex,age_q,Local DI,Local Acc
0,181,0,"(20, 30]",0.075,0.619
1,105,0,"(30, 40]",0.044,0.667
2,60,0,"(40, 50]",0.055,0.733
3,798,1,"(20, 30]",0.010,0.607
4,403,1,"(30, 40]",0.007,0.618
5,223,1,"(40, 50]",0.125,0.677
6,148,1,"(50, 60]",0.190,0.757


 # ADEL MODEL

In [107]:
lambda_=15
modelADEL = ADEL(learning_rate=0.0001,batch_size= 512,
            lamb= lambda_, num_epochs=200,
           NN_y= NN_y_adel, NN_s= NN_s_adel, NN_z=NN_z_adel, GPU=dev)
modelADEL.fit(X_train, y_train, S_train)
y_pred = modelADEL.predict(X_test)
torch.save(modelADEL,'./saved_models/ADEL_'+ str(lambda_))

epoch 0 loss 0.69011503 lossS 0.5590419 lossY 0.69011503 P-rule -100.0 ACC_train 0.5453660909754318
epoch 5 loss 0.69147587 lossS 0.6231382 lossY 0.69147587 P-rule 78.04737975592246 ACC_train 0.553879834590124
epoch 10 loss 0.6531713 lossS 0.5342072 lossY 0.6531713 P-rule 68.86533507875512 ACC_train 0.649720262709803
epoch 15 loss 0.5219346 lossS 0.3327345 lossY 0.5219346 P-rule 70.93767155103248 ACC_train 0.6808562393578205
epoch 20 loss 0.46668324 lossS 0.34300607 lossY 0.46668324 P-rule 68.43016708457657 ACC_train 0.6956944782291413
epoch 25 loss 0.57474893 lossS 0.51655245 lossY 0.57474893 P-rule 66.79459639866919 ACC_train 0.7032352225735831
epoch 30 loss 0.4013372 lossS 0.26025552 lossY 0.4013372 P-rule 63.92062889508983 ACC_train 0.711992216005838
epoch 35 loss 0.51142037 lossS 0.5277404 lossY 0.51142037 P-rule 65.60259553798677 ACC_train 0.7132084650936512
epoch 40 loss 0.6716101 lossS 0.49894387 lossY 0.6716101 P-rule 64.30039809436794 ACC_train 0.7136949647287765
epoch 45 los

In [108]:
#model=torch.load('NN_biased')
#model=torch.load('ZHANG_20')
#model=torch.load('ZHANG_25')
model=torch.load('./saved_models/ADEL_15')

y_pred = model.predict(X_test)
Res = display_results(y_pred, y_test, S_test)
gr = get_subroups(X_test0, S_test, continuous_names=["age"], how_continuous='bins', n_min=N_MIN_GROUP, n_q=10,
             to_keep=["age"]+["sex"])
from utils.fairness_utils import get_subroups_results, get_scores, get_subroups, get_subgroups_random
dro_scores, sr = get_scores(gr[0], X_test0, y_test, S_test, y_pred>0.5, n_min=N_MIN_GROUP,
              topK_DI=[1, 3], q_DI=10, topK_acc=[1], q_acc=4)
print(dro_scores)
sr.drop(["group_label","Acc by S"], axis=1)[['n_obs','sex','age_q','Local DI','Local Acc']]

Accuracy: 0.6371595330739299
PRULE :  91.8956443963724
DI :  0.032681760180677644
DispFPR :  0.023515003061849338
DispFNR :  0.053137774801678306
coucou
{'Global Acc': 0.6371595330739299, 'Global DI': 0.032681760180677644, 'top1_DI': 0.097, 'top3_DI': 0.08333333333333333, 'worst1_acc': 0.614, 'q_DI_0.0': 0.006, 'q_DI_0.1': 0.0066, 'q_DI_0.2': 0.0074, 'q_DI_0.30000000000000004': 0.0086, 'q_DI_0.4': 0.025400000000000013, 'q_DI_0.5': 0.05, 'q_DI_0.6000000000000001': 0.06140000000000002, 'q_DI_0.7000000000000001': 0.07200000000000001, 'q_DI_0.8': 0.08100000000000002, 'q_DI_0.9': 0.08920000000000002, 'Local DI': array([0.007, 0.069, 0.084, 0.009, 0.006, 0.05 , 0.097])}


,n_obs,sex,age_q,Local DI,Local Acc
0,181,0,"(20, 30]",0.007,0.624
1,105,0,"(30, 40]",0.069,0.657
2,60,0,"(40, 50]",0.084,0.733
3,798,1,"(20, 30]",0.009,0.615
4,403,1,"(30, 40]",0.006,0.625
5,223,1,"(40, 50]",0.050,0.614
6,148,1,"(50, 60]",0.097,0.770


# GRARI MODEL (2020)

In [109]:
from models.grari import GRARI
lambda_=0.3
dev= "cuda:0"
modelGRARI = GRARI(regressor="bce",mod_h=NN_y,  p_device =dev,
                                nbepoch =1500 , lr = 0.0005, lambdaHGR=lambda_, nbepochHGR=50, start_epochHGR=1000
                                  ,mod_HGR_F=Net_HGR,mod_HGR_G= Net2_HGR,init_HGR=True)

modelGRARI.fit(X_train, y_train, S_train)
y_pred = modelGRARI.predict_after_proba(X_test)
torch.save(modelGRARI,'./saved_models/GRARI_'+ str(lambda_))

Bar desc (file 1500): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1501/1501 [01:12<00:00, 20.63it/s]


In [110]:
#model=torch.load('NN_biased')
#model=torch.load('ZHANG_20')
#model=torch.load('ZHANG_25')
model=torch.load('./saved_models/GRARI_0.3')

y_pred = model.predict_after_proba(X_test)
Res = display_results(y_pred, y_test, S_test)
gr = get_subroups(X_test0, S_test, continuous_names=["age"], how_continuous='bins', n_min=N_MIN_GROUP, n_q=10,
             to_keep=["age"]+["sex"])
from utils.fairness_utils import get_subroups_results, get_scores, get_subroups, get_subgroups_random
dro_scores, sr = get_scores(gr[0], X_test0, y_test, S_test, y_pred>0.5, n_min=N_MIN_GROUP,
              topK_DI=[1, 3], q_DI=10, topK_acc=[1], q_acc=4)
print(dro_scores)
sr.drop(["group_label","Acc by S"], axis=1)[['n_obs','sex','age_q','Local DI','Local Acc']]

Accuracy: 0.6181906614785992
PRULE :  85.31239799804156
DI :  0.06793968605798417
DispFPR :  0.016084915288834434
DispFNR :  0.1326770314128492
coucou
{'Global Acc': 0.6181906614785992, 'Global DI': 0.06793968605798417, 'top1_DI': 0.154, 'top3_DI': 0.13366666666666668, 'worst1_acc': 0.569, 'q_DI_0.0': 0.009, 'q_DI_0.1': 0.015, 'q_DI_0.2': 0.025400000000000006, 'q_DI_0.30000000000000004': 0.04460000000000001, 'q_DI_0.4': 0.05820000000000001, 'q_DI_0.5': 0.069, 'q_DI_0.6000000000000001': 0.08700000000000002, 'q_DI_0.7000000000000001': 0.10880000000000001, 'q_DI_0.8': 0.13820000000000002, 'q_DI_0.9': 0.1504, 'Local DI': array([0.148, 0.069, 0.051, 0.019, 0.009, 0.154, 0.099])}


,n_obs,sex,age_q,Local DI,Local Acc
0,181,0,"(20, 30]",0.148,0.569
1,105,0,"(30, 40]",0.069,0.610
2,60,0,"(40, 50]",0.051,0.750
3,798,1,"(20, 30]",0.019,0.608
4,403,1,"(30, 40]",0.009,0.608
5,223,1,"(40, 50]",0.154,0.587
6,148,1,"(50, 60]",0.099,0.723


# FELDMAN MODEL

In [91]:
algorithm = FeldmanAlgorithm(DecisionTree())  # Kamishima algorithm
table = [0,0,0,0]
params = {'lambda': 0.9999} 
predicted, params, predictions_list = run_alg(algorithm, X_train, y_train, S_train,X_test,y_test, S_test,params)
y_pred = np.asarray(predicted)
Res = display_results(y_pred, y_test, S_test)
table = np.vstack([table,[Res['Accuracy']*100,Res['PRULE'],Res['DispFPR'],Res['DispFNR']]])
#np.savetxt(sys.stdout, np.mean(table[1:,], axis=0).astype(float), '%5.2f')

gr = get_subroups(X_test0, S_test, continuous_names=["age"], how_continuous='bins', n_min=N_MIN_GROUP, n_q=10,
             to_keep=["age"]+["sex"])
from utils.fairness_utils import get_subroups_results, get_scores, get_subroups, get_subgroups_random
dro_scores, sr = get_scores(gr[0], X_test0, y_test, S_test, y_pred>0.5, n_min=N_MIN_GROUP,
              topK_DI=[1, 3], q_DI=10, topK_acc=[1], q_acc=4)
print(dro_scores)
sr.drop(["group_label","Acc by S"], axis=1)[['n_obs','sex','age_q','Local DI','Local Acc']]

Accuracy: 0.6074902723735408
PRULE :  81.72918962877695
DI :  0.08221187468741581
DispFPR :  0.06868748724229434
DispFNR :  0.05821394434574112
coucou
{'Global Acc': 0.6074902723735408, 'Global DI': 0.08221187468741581, 'top1_DI': 0.099, 'top3_DI': 0.09333333333333334, 'worst1_acc': 0.571, 'q_DI_0.0': 0.003, 'q_DI_0.1': 0.021000000000000005, 'q_DI_0.2': 0.034600000000000006, 'q_DI_0.30000000000000004': 0.039400000000000004, 'q_DI_0.4': 0.05020000000000001, 'q_DI_0.5': 0.064, 'q_DI_0.6000000000000001': 0.07840000000000001, 'q_DI_0.7000000000000001': 0.089, 'q_DI_0.8': 0.092, 'q_DI_0.9': 0.0954, 'Local DI': array([0.041, 0.033, 0.088, 0.003, 0.064, 0.093, 0.099])}


,n_obs,sex,age_q,Local DI,Local Acc
0,181,0,"(20, 30]",0.041,0.591
1,105,0,"(30, 40]",0.033,0.600
2,60,0,"(40, 50]",0.088,0.650
3,798,1,"(20, 30]",0.003,0.571
4,403,1,"(30, 40]",0.064,0.593
5,223,1,"(40, 50]",0.093,0.655
6,148,1,"(50, 60]",0.099,0.730


# KAMISHIMA MODEL

In [92]:
algorithm = KamishimaAlgorithm()  # Kamishima algorithm
table = [0,0,0,0]
params = {'eta': 100} 
predicted, params, predictions_list = run_alg(algorithm, X_train, y_train, S_train,X_test,y_test, S_test,params)
y_pred = np.asarray(predicted)
Res = display_results(y_pred, y_test, S_test)
table = np.vstack([table,[Res['Accuracy']*100,Res['PRULE'],Res['DispFPR'],Res['DispFNR']]])
#np.savetxt(sys.stdout, np.mean(table[1:,], axis=0).astype(float), '%5.2f')

gr = get_subroups(X_test0, S_test, continuous_names=["age"], how_continuous='bins', n_min=N_MIN_GROUP, n_q=10,
             to_keep=["age"]+["sex"])
from utils.fairness_utils import get_subroups_results, get_scores, get_subroups, get_subgroups_random
dro_scores, sr = get_scores(gr[0], X_test0, y_test, S_test, y_pred>0.5, n_min=N_MIN_GROUP,
              topK_DI=[1, 3], q_DI=10, topK_acc=[1], q_acc=4)
print(dro_scores)
sr.drop(["group_label","Acc by S"], axis=1)[['n_obs','sex','age_q','Local DI','Local Acc']]

Accuracy: 0.6371595330739299
PRULE :  89.04290429042904
DI :  0.04264251480744813
DispFPR :  0.034415186772810746
DispFNR :  0.09181386658314383
coucou
{'Global Acc': 0.6371595330739299, 'Global DI': 0.04264251480744813, 'top1_DI': 0.212, 'top3_DI': 0.12333333333333334, 'worst1_acc': 0.609, 'q_DI_0.0': 0.01, 'q_DI_0.1': 0.016600000000000004, 'q_DI_0.2': 0.023800000000000005, 'q_DI_0.30000000000000004': 0.032200000000000006, 'q_DI_0.4': 0.037000000000000005, 'q_DI_0.5': 0.04, 'q_DI_0.6000000000000001': 0.0442, 'q_DI_0.7000000000000001': 0.05980000000000001, 'q_DI_0.8': 0.09820000000000005, 'q_DI_0.9': 0.15140000000000003, 'Local DI': array([0.047, 0.01 , 0.212, 0.111, 0.021, 0.035, 0.04 ])}


,n_obs,sex,age_q,Local DI,Local Acc
0,181,0,"(20, 30]",0.047,0.663
1,105,0,"(30, 40]",0.010,0.676
2,60,0,"(40, 50]",0.212,0.717
3,798,1,"(20, 30]",0.111,0.609
4,403,1,"(30, 40]",0.021,0.620
5,223,1,"(40, 50]",0.035,0.659
6,148,1,"(50, 60]",0.040,0.703


# ZAFAR FAIRNESS MODEL

In [96]:
from models.ZafarAlgorithm import ZafarAlgorithmFairnessNEW
algorithm = ZafarAlgorithmFairnessNEW()  # Kamishima algorithm
table = [0,0,0,0]
params = {'c': 1000000}
predicted, params, predictions_list = run_alg(algorithm, X_train, y_train, S_train,X_test,y_test, S_test,params)
y_pred = np.asarray(predicted)
Res = display_results(y_pred, y_test, S_test)
table = np.vstack([table,[Res['Accuracy']*100,Res['PRULE'],Res['DispFPR'],Res['DispFNR']]])
#np.savetxt(sys.stdout, np.mean(table[1:,], axis=0).astype(float), '%5.2f')

gr = get_subroups(X_test0, S_test, continuous_names=["age"], how_continuous='bins', n_min=N_MIN_GROUP, n_q=10,
             to_keep=["age"]+["sex"])
from utils.fairness_utils import get_subroups_results, get_scores, get_subroups, get_subgroups_random
dro_scores, sr = get_scores(gr[0], X_test0, y_test, S_test, y_pred>0.5, n_min=N_MIN_GROUP,
              topK_DI=[1, 3], q_DI=10, topK_acc=[1], q_acc=4)
print(dro_scores)
sr.drop(["group_label","Acc by S"], axis=1)[['n_obs','sex','age_q','Local DI','Local Acc']]

Accuracy: 0.6541828793774319
PRULE :  68.19081476699924
DI :  0.14053576277886176
DispFPR :  0.06719738722188201
DispFNR :  0.17841655911307436
coucou
{'Global Acc': 0.6541828793774319, 'Global DI': 0.14053576277886176, 'top1_DI': 0.181, 'top3_DI': 0.16733333333333333, 'worst1_acc': 0.625, 'q_DI_0.0': 0.004, 'q_DI_0.1': 0.0148, 'q_DI_0.2': 0.0232, 'q_DI_0.30000000000000004': 0.0268, 'q_DI_0.4': 0.07360000000000004, 'q_DI_0.5': 0.142, 'q_DI_0.6000000000000001': 0.14679999999999999, 'q_DI_0.7000000000000001': 0.1542, 'q_DI_0.8': 0.16680000000000003, 'q_DI_0.9': 0.17500000000000002, 'Local DI': array([0.028, 0.171, 0.004, 0.022, 0.142, 0.15 , 0.181])}


,n_obs,sex,age_q,Local DI,Local Acc
0,181,0,"(20, 30]",0.028,0.669
1,105,0,"(30, 40]",0.171,0.638
2,60,0,"(40, 50]",0.004,0.700
3,798,1,"(20, 30]",0.022,0.625
4,403,1,"(30, 40]",0.142,0.633
5,223,1,"(40, 50]",0.150,0.686
6,148,1,"(50, 60]",0.181,0.777


# ZAFAR ACCURACY MODEL

In [52]:
from models.ZafarAlgorithm import ZafarAlgorithmAccuracyNEW
algorithm = ZafarAlgorithmAccuracyNEW()  # Kamishima algorithm
table = [0,0,0,0]
params = {'gamma': 0.8} 
predicted, params, predictions_list = run_alg(algorithm, X_train, y_train, S_train,X_test,y_test, S_test,params)
y_pred = np.asarray(predicted)
Res = display_results(y_pred, y_test, S_test)
table = np.vstack([table,[Res['Accuracy']*100,Res['PRULE'],Res['DispFPR'],Res['DispFNR']]])
#np.savetxt(sys.stdout, np.mean(table[1:,], axis=0).astype(float), '%5.2f')

gr = get_subroups(X_test0, S_test, continuous_names=["age"], how_continuous='bins', n_min=N_MIN_GROUP, n_q=10,
             to_keep=["age"]+["sex"])
from utils.fairness_utils import get_subroups_results, get_scores, get_subroups, get_subgroups_random
dro_scores, sr = get_scores(gr[0], X_test0, y_test, S_test, y_pred>0.5, n_min=N_MIN_GROUP,
              topK_DI=[1, 3], q_DI=10, topK_acc=[1], q_acc=4)
print(dro_scores)
sr.drop(["group_label","Acc by S"], axis=1)[['n_obs','sex','age_q','Local DI','Local Acc']]

Accuracy: 0.5797665369649806
PRULE :  47.60501606127996
DI :  0.3557226632331771
DispFPR :  0.42020820575627676
DispFNR :  0.3082638960141269
coucou
{'Global Acc': 0.5797665369649806, 'Global DI': 0.3557226632331771, 'top1_DI': 0.469, 'top3_DI': 0.4486666666666666, 'worst1_acc': 0.529, 'q_DI_0.0': 0.117, 'q_DI_0.1': 0.16860000000000003, 'q_DI_0.2': 0.22840000000000005, 'q_DI_0.30000000000000004': 0.30460000000000004, 'q_DI_0.4': 0.342, 'q_DI_0.5': 0.36, 'q_DI_0.6000000000000001': 0.40440000000000004, 'q_DI_0.7000000000000001': 0.4358, 'q_DI_0.8': 0.44120000000000004, 'q_DI_0.9': 0.4534, 'Local DI': array([0.443, 0.36 , 0.469, 0.434, 0.33 , 0.117, 0.203])}


,n_obs,sex,age_q,Local DI,Local Acc
0,181,0,"(20, 30]",0.443,0.619
1,105,0,"(30, 40]",0.360,0.619
2,60,0,"(40, 50]",0.469,0.617
3,798,1,"(20, 30]",0.434,0.564
4,403,1,"(30, 40]",0.330,0.608
5,223,1,"(40, 50]",0.117,0.529
6,148,1,"(50, 60]",0.203,0.608


# CALDERS MODEL

In [98]:
algorithm = CaldersAlgorithm()  # Kamishima algorithm
table = [0,0,0,0]
params = {'beta': 0.2}
predicted, params, predictions_list = run_alg(algorithm, X_train, y_train, S_train,X_test,y_test, S_test,params)
y_pred = np.asarray(predicted)
Res = display_results(y_pred, y_test, S_test)
table = np.vstack([table,[Res['Accuracy']*100,Res['PRULE'],Res['DispFPR'],Res['DispFNR']]])
#np.savetxt(sys.stdout, np.mean(table[1:,], axis=0).astype(float), '%5.2f')

gr = get_subroups(X_test0, S_test, continuous_names=["age"], how_continuous='bins', n_min=N_MIN_GROUP, n_q=10,
             to_keep=["age"]+["sex"])
from utils.fairness_utils import get_subroups_results, get_scores, get_subroups, get_subgroups_random
dro_scores, sr = get_scores(gr[0], X_test0, y_test, S_test, y_pred>0.5, n_min=N_MIN_GROUP,
              topK_DI=[1, 3], q_DI=10, topK_acc=[1], q_acc=4)
print(dro_scores)
sr.drop(["group_label","Acc by S"], axis=1)[['n_obs','sex','age_q','Local DI','Local Acc']]

Accuracy: 0.6376459143968871
PRULE :  87.92913524400497
DI :  0.05825154155784107
DispFPR :  0.004898958971218614
DispFNR :  0.0725406228568034
coucou
{'Global Acc': 0.6376459143968871, 'Global DI': 0.05825154155784107, 'top1_DI': 0.28, 'top3_DI': 0.20233333333333337, 'worst1_acc': 0.602, 'q_DI_0.0': 0.004, 'q_DI_0.1': 0.011200000000000002, 'q_DI_0.2': 0.02480000000000001, 'q_DI_0.30000000000000004': 0.05120000000000001, 'q_DI_0.4': 0.09680000000000002, 'q_DI_0.5': 0.152, 'q_DI_0.6000000000000001': 0.1544, 'q_DI_0.7000000000000001': 0.159, 'q_DI_0.8': 0.168, 'q_DI_0.9': 0.21460000000000007, 'Local DI': array([0.28 , 0.06 , 0.004, 0.016, 0.152, 0.171, 0.156])}


,n_obs,sex,age_q,Local DI,Local Acc
0,181,0,"(20, 30]",0.280,0.602
1,105,0,"(30, 40]",0.060,0.676
2,60,0,"(40, 50]",0.004,0.700
3,798,1,"(20, 30]",0.016,0.614
4,403,1,"(30, 40]",0.152,0.608
5,223,1,"(40, 50]",0.171,0.673
6,148,1,"(50, 60]",0.156,0.730
